In [ ]:
!ls ../../..

In [ ]:
!pip install tensorflow-object-detection-api

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
!git clone https://github.com/tensorflow/models

In [ ]:
%cd /kaggle/working/
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz
!tar -xvf ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz
!rm ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz

In [ ]:
%cd /kaggle/working/models/research
!protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .
!pip install tensorflow==2.13.0

In [ ]:
!python /kaggle/working/models/research/object_detection/builders/model_builder_tf2_test.py

In [ ]:
!pip install wget

In [ ]:
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.config
!mv ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.config /kaggle/working/ssd_resnet101.config

In [ ]:
num_classes = 12
batch_size = 4
num_steps = 20000
num_eval_steps = 1000

train_record_path = '/kaggle/input/tfrecord/train.tfrecord'
test_record_path = '/kaggle/input/tfrecord/test.tfrecord'
model_dir = '/kaggle/working/training/'
labelmap_path = '/kaggle/input/labelmap/labelmap.pbtxt'

pipeline_config_path = '/kaggle/working/ssd_resnet101.config'
fine_tune_checkpoint = '/kaggle/working/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0'

In [ ]:
import re

with open(pipeline_config_path) as f:
    config = f.read()

with open(pipeline_config_path, 'w') as f:

    # Set labelmap path
    config = re.sub('label_map_path: ".*?"',
                  'label_map_path: "{}"'.format(labelmap_path), config)

    # Set fine_tune_checkpoint path
    config = re.sub('fine_tune_checkpoint: ".*?"',
                  'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), config)

    # Set fine_tune_checkpoint typr
    config = re.sub('fine_tune_checkpoint_type: ".*?"',
                  'fine_tune_checkpoint_type: "{}"'.format('detection'), config)

    # Set train tf-record file path
    config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")',
                  'input_path: "{}"'.format(train_record_path), config)

    # Set test tf-record file path
    config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")',
                  'input_path: "{}"'.format(test_record_path), config)

    # Set number of classes.
    config = re.sub('num_classes: [0-9]+',
                  'num_classes: {}'.format(num_classes), config)

    # Set batch size
    config = re.sub('batch_size: [0-9]+',
                  'batch_size: {}'.format(batch_size), config)

    # Set training steps
    config = re.sub('num_steps: [0-9]+',
                  'num_steps: {}'.format(num_steps), config)

    # Set learning rate
   # config = re.sub('learning_rate_base: [.0-9]+',
    #              'learning_rate_base: {}'.format(learning_rate), config)

    f.write(config)

In [ ]:
import re

# Hapus seluruh blok konfigurasi optimizer momentum
config = re.sub(r'optimizer\s*{[^}]*}', '', config)

    # Pengaturan optimizer Adam yang akan digunakan
rms_prop_optimizer_config = """
    optimizer {
      rms_prop_optimizer: {
        learning_rate: {
          cosine_decay_learning_rate {
            learning_rate: 0.001  # Sesuaikan dengan kebutuhan Anda
          }
        }
      }
      use_moving_average: false
    }
    """

    # Gabungkan pengaturan optimizer Adam dengan konfigurasi yang ada
config += rms_prop_optimizer_config

print(config)

In [ ]:
!python /kaggle/working/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    #--checkpoint_every_n=100 \
    --num_eval_steps={num_eval_steps}

In [ ]:
!python /kaggle/working/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --checkpoint_dir={model_dir}

In [ ]:
output_directory = '/kaggle/working/inference_graph'

!python /kaggle/working/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir={model_dir} \
    --output_directory={output_directory} \
    --pipeline_config_path={pipeline_config_path}

In [ ]:
!python /kaggle/working/models/research/object_detection/export_tflite_graph_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --trained_checkpoint_dir /kaggle/working/training \
    --output_directory /kaggle/working/inference_graph_tflite

In [ ]:
%cd /kaggle/working/inference_graph_tflite
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model('/kaggle/working/inference_graph_tflite/saved_model') # path to the SavedModel directory
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()
# Save the TensorFlow Lite model to a file
with open('model17.tflite', 'wb') as f:
    f.write(tflite_model)